## Purpose:

This notebook will begin as a walkthrough of my data cleaning, resulting in function(s) that can be called in a .py script to clean the fangraphs data and prep it for analysis.

In [1]:
import os
import sys

import pandas as pd
import numpy as np

sys.path.append('/Users/John/Documents/allProjects/genericfunctions')

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor

from scipy.stats import randint

from FBB_points import *
import warnings
warnings.filterwarnings("ignore")

seedA = 73

### Import data and take a head


#### *Batters*
A lot of these columns won't apply to fantasy baseball. There are a few projected points columns, but that won't work for leagues with custom scoring, as such we need to drop them, all of the columns of NaNs and some of the advanced stats that don't count for scoring.

It's worth noting that some of these stats apply to categories leagues, but this tool is designed specifically for points, so I'm going to drop OBP, OPS, etc.

In [2]:
batters = pd.read_csv(vars['data_raw'] + vars['batters_filename'])
print(batters.columns)
batters.head()

Index(['Name', 'Team', 'G', 'PA', 'AB', 'H', '1B', '2B', '3B', 'HR', 'R',
       'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH', 'GDP', 'SB', 'CS', 'AVG',
       'BB%', 'K%', 'BB/K', 'OBP', 'SLG', 'wOBA', 'OPS', 'ISO', 'Spd', 'BABIP',
       'UBR', 'wSB', 'wRC', 'wRAA', 'wRC+', 'BsR', 'Fld', 'Off', 'Def', 'WAR',
       'ADP', 'InterSD', 'InterSK', 'IntraSD', 'Vol', 'Skew', 'Dim', 'FPTS',
       'FPTS/G', 'SPTS', 'SPTS/G', 'P10', 'P20', 'P30', 'P40', 'P50', 'P60',
       'P70', 'P80', 'P90', 'TT10', 'TT20', 'TT30', 'TT40', 'TT50', 'TT60',
       'TT70', 'TT80', 'TT90', 'NameASCII', 'PlayerId', 'MLBAMID'],
      dtype='object')


,Name,Team,G,PA,AB,H,1B,2B,3B,HR,...,TT30,TT40,TT50,TT60,TT70,TT80,TT90,NameASCII,PlayerId,MLBAMID
0,Aaron Judge,NYY,159,686,559,158,80,28,0,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaron Judge,15640,592450
1,Juan Soto,NYM,159,686,539,149,87,27,2,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juan Soto,20123,665742
2,Shohei Ohtani,LAD,151,651,564,164,84,32,5,43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shohei Ohtani,19755,660271
3,Bobby Witt Jr.,KCR,159,686,623,180,105,37,8,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bobby Witt Jr.,25764,677951
4,Gunnar Henderson,BAL,157,679,593,163,97,30,7,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gunnar Henderson,26289,683002


In [3]:
batters = batters[['Name', 'Team', 'PlayerId', 'AB', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'SO', 'SB', 'CS']]

### Key Finding - No Position

There is no field listing a player's position. Most leagues have specific positional rules for batters, such as one C, 1B, 2B, 3B, and SS in addition to 3 OF. There may also be a DH slot, and a few utility spots. As such, we need to get positional information.

The eligibility rules vary by platform, which adds complexity. Without accessing each platform's dataset, I won't be able to adjust to ESPN, CBS, Yahoo, etc., therefore I'm going to focus on the positions as listed by ZiPS.

To access these datapoints, I need to download one file per position. It's not ideal, but there is no automated workaround.

With one file per position in hand, I loaded them in and printed the shape to verify the contain the same fields. It seems that they do, which is excellent.

I'm going to need to iteratively append these and update records in the event of duplicates. Players can be eligible for multiple positions, but I want one row per player. This will also need to take place in the case of Shohei Ohtani, because he will be both a batter and a pitcher.

In [4]:
batterTypes = ['C' 
              ,'1B'
              ,'2B'
              ,'3B'
              ,'SS'
              ,'OF'
              ,'DH'
              ]

for pos in batterTypes:
    battersTemp = pd.read_csv(vars['data_raw'] + vars['filename'].format(pos=pos))
    print(pos, battersTemp.shape)
    
    #battersTemp['POS'] = pos
    #battersTemp['PlayerId'] = battersTemp['PlayerId'].astype(str)
    
    # if (battersAll.shape[0] != 0):
    #     battersAll = updateDataFrame(battersTemp, battersAll, pos)
        
    # elif (battersAll.shape[0] == 0):
    #     battersAll = pd.concat([battersAll, battersTemp])

C (100, 74)
1B (70, 74)
2B (86, 74)
3B (88, 74)
SS (102, 74)
OF (265, 74)
DH (82, 74)


In [5]:
def updateDataFrame(new, total, pos):
   """
   This function identifies the players in the current dataset, and the dataset containing the new position.
      -If any players are present in both, we append the new position to their POS column.
      -If they are present in only the new dataset, we append the record and change nothing.
      -If they are present in only the old dataset, we simply leave them alone.
      -Duplicate records are dropped.
   """
   # Identify the players in both records via inner join
   overlap = pd.merge(total,new, how = 'inner', on= ['Name','Team'], suffixes=('', '_New'))
   ids = overlap['PlayerId'].tolist()
    
   # Identify players from large DF in new dataframe - update required
   ## These are IN BOTH new and total and must be dropped from BOTH to prevent duplicates
   update = total[total['PlayerId'].isin(ids)]
    
   # Identify the players not in new dataframe - no update required
   noUpdate = total[~(total['PlayerId'].isin(ids))]
   
   # drop updated records from new dataframe
   dropRecords = new[~(new['PlayerId'].isin(ids))]
    
   # Update the POS column for additional Positions
   update['POS'] = update['POS'] + f", {pos}".format(pos = pos)
    
   # Concatenate all three dfs
   finalProduct = pd.concat([update, noUpdate, dropRecords]).reset_index(drop = True)
    
   return(finalProduct)   

In [6]:
def cleanAddPOS(positions, filepath, filename):
    outputAll = pd.DataFrame()

    for pos in positions:
        ## load in the position-specific csv
        posTemp = pd.read_csv(vars['data_raw'] + vars['filename'].format(pos=pos))
        
        ## set the POS to the corresponding position
        posTemp['POS'] = pos
        ## make the PlayerID a string to prevent any strange integer operations
        posTemp['PlayerId'] = posTemp['PlayerId'].astype(str)
        
        ## if there are multiple rows, update the dataframe with our previous defined function
        if (outputAll.shape[0] != 0):
            outputAll = updateDataFrame(posTemp, outputAll, pos)
        
        ## if there are zero rows, then concat the data to the df
        else:
            outputAll = pd.concat([outputAll, posTemp])
    return outputAll

In [7]:
cleanAddPOS(batterTypes, vars['data_raw'], vars['filename']).head()

,Name,Team,G,PA,AB,H,1B,2B,3B,HR,...,TT40,TT50,TT60,TT70,TT80,TT90,NameASCII,PlayerId,MLBAMID,POS
0,Cooper Hummel,HOU,3,14,12,3,2,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Cooper Hummel,19458,669450,"C, OF, DH"
1,Mark Canha,MIL,36,154,131,32,22,7,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,Mark Canha,11445,592192,"1B, OF, DH"
2,Juan Yepez,WSN,23,98,89,22,14,5,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,Juan Yepez,18400,660766,"1B, OF, DH"
3,Trey Mancini,ARI,3,14,13,3,2,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Trey Mancini,15149,641820,"1B, OF, DH"
4,Kris Bryant,COL,110,476,420,102,68,22,0,12,...,NaN,NaN,NaN,NaN,NaN,NaN,Kris Bryant,15429,592178,"1B, OF, DH"


In [8]:
## Initialize an empty dataframe for the final output
battersAll = pd.DataFrame()

## List all batter positions
batterTypes = ['C' ,
               '1B',
               '2B',
               '3B',
               'SS',
               'OF',
               'DH'
              ]


### this is lovely but it needs to become a function as we repeat for pitchers
for pos in batterTypes:
    ## load in the position-specific csv
    battersTemp = pd.read_csv(vars['data_raw'] + vars['filename'].format(pos=pos))
    
    ## set the POS to the corresponding position
    battersTemp['POS'] = pos
    ## make the PlayerID a string to prevent any strange integer operations
    battersTemp['PlayerId'] = battersTemp['PlayerId'].astype(str)
    
    ## if there are multiple rows, update the dataframe with our previous defined function
    if (battersAll.shape[0] != 0):
       battersAll = updateDataFrame(battersTemp, battersAll, pos)
    
    ## if there are zero rows, then concat the data to the df
    else:
        battersAll = pd.concat([battersAll, battersTemp])

### Batter output
This gives us 595 eligible batters and 75 columns. Let's select the columns relevant to fantasy baseball.

In [9]:
battersAll.shape

(595, 75)

### Batters output

This gives us exactly the information we need and nothing more. Great stuff, this is what we want to see

In [10]:
battersAll[['Name', 'Team', 'POS', 'PlayerId', 'AB', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'SO', 'SB', 'CS']].head()

,Name,Team,POS,PlayerId,AB,H,1B,2B,3B,HR,R,RBI,SO,SB,CS
0,Cooper Hummel,HOU,"C, OF, DH",19458,12,3,2,1,0,0,2,2,4,0,0
1,Mark Canha,MIL,"1B, OF, DH",11445,131,32,22,7,0,2,17,15,30,2,0
2,Juan Yepez,WSN,"1B, OF, DH",18400,89,22,14,5,0,3,10,12,19,0,0
3,Trey Mancini,ARI,"1B, OF, DH",15149,13,3,2,1,0,0,1,1,4,0,0
4,Kris Bryant,COL,"1B, OF, DH",15429,420,102,68,22,0,12,54,49,117,2,0


### Pitchers

There is little critical thinking required here for the cleaning - it follows identical logic to what we constructed before. We want to select the columns that are necessary for fantasy and for the modeling activity ahead.

In [11]:
pitcherTypes = ['SP' 
              ,'RP'
              ]

pitchersAll = cleanAddPOS(pitcherTypes, vars['data_raw'], vars['filename'])
pitchersAll = pitchersAll[['Name', 'Team', 'PlayerId', 'POS', 'W', 'L', 'G', 'GS', 'SV', 'HLD', 'HR', 'IP', 'H', 'R', 'ER', 'BB', 'SO']]
pitchersAll.set_index('Name', inplace = True)

pitchersAll.head()

,Team,PlayerId,POS,W,L,G,GS,SV,HLD,HR,IP,H,R,ER,BB,SO
Name,,,,,,,,,,,,,,,,
Tarik Skubal,DET,22267,SP,15,8,32,32,0,0,16,198,157,65,60,39,227
Zack Wheeler,PHI,10310,SP,15,8,32,32,0,0,20,203,168,80,74,54,213
Chris Sale,ATL,10603,SP,14,6,31,31,0,0,17,178,151,64,59,46,218
Logan Webb,SFG,17995,SP,13,9,32,32,0,0,14,198,185,77,71,43,166
Garrett Crochet,BOS,27463,SP,13,6,31,31,0,0,12,143,120,51,47,46,179


### A couple of data notes

The Blown Saves column is populated with only NaNs, unfortunately. This is an important stat because many leagues give negative points for a blown save. Leaving it as a zero will make relievers appear more valuable than they are. 

#### Handling Approach:

The goal is to avoid overestimating the value of a relief pitchers, therefore a simple approach should make the adjustment easy.

I am going to apply the following assumptions:
- Blown Saves are a random variable and volatile, making them hard to predict
- An individual save opportunity follows a bernoulli distribution with P(Blown Save) =  p
- While some players, such as Emmanuel Clase, are less likely to blow a save, a constant P(Blown Save) is appropriate
- The more save opportunities a player has, the more blown save opportunities they have

Our estimator will be constructed from 2024 results. The estimated probability of a save will be BS/SV for the entire league. Our estimated BS for each player will be p * SV.

In [12]:
teams = pd.read_csv(vars['data_raw'] + vars['team_pitching'])

In [13]:
## Clearly the ratio is high, this will penalize relievers strongly.
phat = sum(teams['BS'])/sum(teams['SV'])

In [14]:
pitchersAll['BS'] = round(pitchersAll['SV'] * phat)

In [15]:
pitchersAll.sort_values(by = 'BS', ascending = False).head()

,Team,PlayerId,POS,W,L,G,GS,SV,HLD,HR,IP,H,R,ER,BB,SO,BS
Name,,,,,,,,,,,,,,,,,
Ryan Helsley,STL,18138,RP,7,3,66,0,35,2,5,66,48,22,20,24,80,20.0
Edwin Díaz,NYM,14710,RP,7,4,65,0,36,2,6,65,44,22,21,22,96,20.0
Josh Hader,HOU,14212,RP,9,4,68,0,33,2,8,68,46,25,23,25,98,19.0
Emmanuel Clase,CLE,21032,RP,6,1,66,0,34,2,4,66,50,17,16,13,65,19.0
Raisel Iglesias,ATL,17130,RP,6,4,68,0,32,1,7,68,56,23,21,15,77,18.0


#### Thoughts on this method
These predictions appear to be terrible. Projecting 19 BS for Clase is malpractice, but let's confirm these vs last year's performance results.

In [16]:
indivs = pd.read_csv(vars['data_raw'] + vars['indiv_pitching'])

indivs.sort_values(by = 'SV', ascending=False)[['Name','Team','SV', 'BS']].head()

,Name,Team,SV,BS
0,Ryan Helsley,STL,49,4
1,Emmanuel Clase,CLE,47,3
2,Kyle Finnegan,WSN,38,5
3,Robert Suarez,SDP,36,6
4,Josh Hader,HOU,34,4


In [17]:
phat = sum(teams['BS'])/sum(teams['SV'])

indivs['BS_hat'] = indivs['SV'] * phat

## 7.73, which is god awful
print("MSE for Constant p-hat: ",
      round(sum((indivs['BS'] - indivs['BS_hat'])**2)/len(indivs),2))

MSE for Constant p-hat:  7.73


Clearly the previous assumptions were terrible. Being off by 10 is horrible, and the estimate penalizes the best relievers equal to those who generate more blown saves. That clearly doesn't work.

I'm going to fit a decision tree to the 2024 data to estimate blown saves. It's fair to assume that blown saves do not follow a known distribution, in particular not the bernoulli/binomial distribution. As such, we want a model that does not rely upon a known distribution, but can still estimate a continuous distribution. Here come the trees!

In [18]:
indivs.set_index('Name', inplace = True)
indivs.columns
indivs = indivs[['ERA', 'G', 'GS', 'SV', 'HLD', 'BS', 'IP', 'HR', 'BB', 'SO']]

## count the relief apperances and drop anyone with less than 5, as that may cause odd results
indivs['ReliefApps'] = indivs['G'] - indivs['GS']
indivs = indivs[indivs['ReliefApps'] >= 5]
indivs.drop(['G', 'GS'], axis = 1, inplace = True)

In [19]:
indivs.head()

,ERA,SV,HLD,BS,IP,HR,BB,SO,ReliefApps
Name,,,,,,,,,
Ryan Helsley,2.035176,49,0,4,66.1,3,23,79,65
Emmanuel Clase,0.605381,47,0,3,74.1,2,10,66,74
Kyle Finnegan,3.675393,38,0,5,63.2,9,24,60,65
Robert Suarez,2.769231,36,1,6,65.0,7,16,59,65
Josh Hader,3.802817,34,0,4,71.0,12,25,105,71


In [21]:
y = indivs['BS']
X = indivs.drop('BS', axis = 1)

First, overfit a model on all the data to validate that this single tree approach will work to some extent

In [22]:
dt = DecisionTreeRegressor()
dt.fit(X,y)
preds = dt.predict(X)
## MSE
print(sum((y-preds)**2)/len(y))

0.0


It's safe to say the overfit model overfits and predicts well considering that it returned identical predictions. Now we need to tune this this so it doesn't provide completely useless results.

In [23]:
## identify the number of splits
print(dt.tree_.max_depth)


18


In [24]:
## 80-20 train test split for validation
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=seedA, train_size = .8)

### it's a single decision tree so we can search exhaustively here with ease
gridParams = {
    'max_depth': [5, 10],
    'min_samples_split': [5, 10, 20],
    'min_samples_leaf': [5, 10, 25],
    'ccp_alpha': [0.0001, 0.001, 0.01, 0.1]
}
gridTree = DecisionTreeRegressor()
regTree = GridSearchCV(gridTree, gridParams, scoring = 'neg_mean_squared_error')
regTree.fit(Xtrain, ytrain)


GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'ccp_alpha': [0.0001, 0.001, 0.01, 0.1],
                         'max_depth': [5, 10], 'min_samples_leaf': [5, 10, 25],
                         'min_samples_split': [5, 10, 20]},
             scoring='neg_mean_squared_error')

In [25]:
regTree.best_params_

{'ccp_alpha': 0.001,
 'max_depth': 5,
 'min_samples_leaf': 25,
 'min_samples_split': 5}

In [26]:
preds = regTree.predict(Xtest)

In [27]:
round(sum((preds-ytest)**2)/len(ytest),2)

2.35

The average error is 2.35 blown saves. That kind of stinks. Let's try something else

#### Random Forest
Many decision trees might help improve predictions. We will use randomized search to speed this up as I don't feel the need to be exhaustive.

In [28]:
rfParams = {
    'n_estimators': randint(50, 500),  # Randomized selection between 50 and 500
    'max_depth': randint(3, 15),  # Randomized selection between 3 and 15
    'min_samples_split': randint(2, 20),  # Randomized selection between 2 and 20
    'min_samples_leaf': randint(1, 10),  # Randomized selection between 1 and 10
    'ccp_alpha': np.logspace(-4, -1, 10)  # Log-spaced values between 0.0001 and 0.1
}

rf = RandomForestRegressor(random_state=seedA)

# Define RandomizedSearchCV
random_search = RandomizedSearchCV(
    rf, param_distributions=rfParams,
    n_iter=50,  # Number of random combinations to try
    scoring='neg_mean_squared_error',  # Evaluation metric
    cv=5,  # 5-fold cross-validation
    verbose=2,  # Print progress
    n_jobs=-1,  # Use all processors
    random_state=42
)

random_search.fit(Xtrain, ytrain)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=73),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'ccp_alpha': array([0.0001    , 0.00021544, 0.00046416, 0.001     , 0.00215443,
       0.00464159, 0.01      , 0.02154435, 0.04641589, 0.1       ]),
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x126796150>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x126c7b530>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1267cb7a0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x126bd85f0>},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [29]:
predsrf = random_search.best_estimator_.predict(Xtest)


In [30]:
round(sum((predsrf-ytest)**2)/len(ytest),2)

2.21

The performance is marginally better. I'll take it. This absolutely beats the previous method and will at least allow me to make some predictions.

It's time to turn the cleaning and modeling into a script.

In [31]:
Xtest.columns

Index(['ERA', 'SV', 'HLD', 'IP', 'HR', 'BB', 'SO', 'ReliefApps'], dtype='object')

In [32]:
def pitcherDataPrep(data, estimator):
    ## create two necessary features
    data['ERA'] = data['ER']/(data['IP']/9)
    data['ReliefApps'] = data['G'] - data['GS']


    ### only predict for players with 5+ estimated relief appearances
    modelData = data[data['ReliefApps'] >= 5]
    modelData = modelData[['ERA', 'SV', 'HLD', 'IP', 'HR', 'BB', 'SO', 'ReliefApps']]
    
    # use estimator to make predictions
    modelData['BS'] = estimator.predict(modelData)

    ## join to data, making the output dataset
    output = data.merge(modelData[['BS']], how = 'left', left_index = True, right_index = True)
    
    ## fillna with 0 for the 4 or fewer relief apps
    output['BS'].fillna(0, inplace = True)

    ## return only the required columns for fantasy baseball
    return output[['Team', 'POS', 'W', 'IP', 'HLD', 'SV', 'SO', 'ER', 'BS']]
    #return modelData
out = pitcherDataPrep(pitchersAll.drop('BS', axis = 1),random_search.best_estimator_)


In [33]:
out.head()

,Team,POS,W,IP,HLD,SV,SO,ER,BS
Name,,,,,,,,,
Tarik Skubal,DET,SP,15,198,0,0,227,60,0.0
Zack Wheeler,PHI,SP,15,203,0,0,213,74,0.0
Chris Sale,ATL,SP,14,178,0,0,218,59,0.0
Logan Webb,SFG,SP,13,198,0,0,166,71,0.0
Garrett Crochet,BOS,SP,13,143,0,0,179,47,0.0
